In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
ACCESS_KEY = "yuQTUA75jptY1N30aLfb"
SECRET_KEY = "Lz6ajCsZydsIaHCOuAoCUc8VbK5G70nyQsHeIDpq"
MINIO_URL = "http://minio:9000"

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("HW2") \
    .config("spark.sql.adaptive.enabled", False) \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.sql.sources.bucketing.enabled", True) \
    .config("spark.sql.legacy.allowNonEmptyLocationInCTAS", True)\
    .config("spark.executor.memory", "600M") \
    .config("spark.driver.memory", "600M") \
    .config('spark.jars.packages',
        "org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-pom:1.12.365,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1"
    ) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.access.key", ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_URL) \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.11/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-pom added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d94d632f-f263-4c1e-b4fd-840adeb5cb41;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found com.amazonaws#aws-java-sdk-pom;1.12.365 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#ha

In [3]:
spark

# Задание 1

## Входные данные
- Файл с данными по оттоку телеком оператора в США (churn.csv)
- Справочник с названиями штатов (state.json)
- Справочник с численностью населения территорий (определяется полем area code) внутри штатов (state.json)
- Террия с численностью населения меньше 10_000 считается **мелкой**

## Что нужно сделать
1. Посчитать количество отточных и неотточных абонентов (поле churn), исключив **мелкие** территории
2. Отчет должен быть выполнен в разрезе **каждого штата** с его полным наименованием
3. Описать возникающие узкие места при выполнении данной операции
4. Применить один из способов оптимизации для ускорения выполнения запроса (при допущении, что справочник численности населения **сильно меньше** основных данных)
5. Если существует еще какой-то способ, применить также и его отдельно от п.4 (при допущении, что справочник численности населения **сопоставим по размеру** с основными данными)
6. Кратко описать реализованные способы и в чем их практическая польза

- P.S. Одним из выбранных способов должен быть Bucket specific join
- P.P.S. При обосновании предлагаем прикладывать запуска команды df.explain()

In [4]:
churn_df = spark.read.option("header", True).csv("s3a://input/data/churn.csv")
state_dict = spark.read.json("s3a://input/data/state.json")
pop_dict = spark.read.json("s3a://input/data/population.json")

24/06/27 20:02:55 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


### Решение

In [5]:
churn_df, state_dict, pop_dict

(DataFrame[id: string, state: string, account length: string, area code: string, phone number: string, international plan: string, voice mail plan: string, number vmail messages: string, total day minutes: string, total day calls: string, total day charge: string, total eve minutes: string, total eve calls: string, total eve charge: string, total night minutes: string, total night calls: string, total night charge: string, total intl minutes: string, total intl calls: string, total intl charge: string, customer service calls: string, churn: string, val: string],
 DataFrame[state_id: string, state_name: string],
 DataFrame[area code: bigint, population: bigint])

In [7]:
filtered_churn = churn_df.join(pop_dict, on="area code", how="inner") \
    .filter(F.col("population") > 10000)

result = filtered_churn.join(state_dict, filtered_churn.state==state_dict.state_id, how="left") \
    .groupBy("state_name") \
    .pivot("churn") \
    .agg(F.count("*")) \
    .orderBy("state_name")

In [8]:
# Здесь необходимо вывести результат:
result.explain()
result.show(truncate=False)

== Physical Plan ==
*(12) Sort [state_name#72 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(state_name#72 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=307]
   +- *(11) Project [state_name#72, __pivot_count(1) AS `count(1)`#207[0] AS False#208L, __pivot_count(1) AS `count(1)`#207[1] AS True#209L]
      +- HashAggregate(keys=[state_name#72], functions=[pivotfirst(churn#38, count(1)#201L, False, True, 0, 0)])
         +- Exchange hashpartitioning(state_name#72, 200), ENSURE_REQUIREMENTS, [plan_id=302]
            +- HashAggregate(keys=[state_name#72], functions=[partial_pivotfirst(churn#38, count(1)#201L, False, True, 0, 0)])
               +- *(10) HashAggregate(keys=[state_name#72, churn#38], functions=[count(1)])
                  +- Exchange hashpartitioning(state_name#72, churn#38, 200), ENSURE_REQUIREMENTS, [plan_id=297]
                     +- *(9) HashAggregate(keys=[state_name#72, churn#38], functions=[partial_count(1)])
                        +- *(9) Project [

+--------------------+-----+----+
|state_name          |False|True|
+--------------------+-----+----+
|Alabama             |58   |7   |
|Alaska              |44   |3   |
|Arizona             |53   |4   |
|Arkansas            |35   |9   |
|California          |21   |8   |
|Colorado            |49   |7   |
|Connecticut         |56   |8   |
|Delaware            |48   |8   |
|District of Columbia|43   |4   |
|Florida             |45   |8   |
|Georgia             |42   |8   |
|Hawaii              |42   |3   |
|Idaho               |53   |9   |
|Illinois            |49   |5   |
|Indiana             |59   |9   |
|Iowa                |34   |3   |
|Kansas              |50   |12  |
|Kentucky            |51   |8   |
|Louisiana           |47   |4   |
|Maine               |49   |13  |
+--------------------+-----+----+
only showing top 20 rows



**Узкие места**: 
1. Перекос данных в файлах state.json и population.json, поэтому некоторые рабочие узлы могут получать больше данных, что замедлит выполнение операций.
2. Можно использовать broadcast has join, так как у нас небольшие файлы (< 10 мб).

### Оптимизация 1

Используем **broadcast has join**: с помощью него данные будут распределены локально на все рабочие узлы, также данные из обоих датасетов при соединении не нужно будет перетасовывать (так как будет перетасовываться только бОльший датасет)

In [9]:
broadcasted_pop_dict = F.broadcast(pop_dict)
broadcasted_state_dict = F.broadcast(state_dict)

broadcasted_filtered_churn = churn_df.join(broadcasted_pop_dict, on="area code", how="inner") \
    .filter(F.col("population") > 10000)

result = broadcasted_filtered_churn.join(broadcasted_state_dict, broadcasted_filtered_churn.state==state_dict.state_id, how="left") \
    .groupBy("state_name") \
    .pivot("churn") \
    .agg(F.count("*")) \
    .orderBy("state_name")

In [10]:
# Здесь необходимо вывести результат:
result.explain()
result.show(truncate=False)

== Physical Plan ==
*(6) Sort [state_name#72 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(state_name#72 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=826]
   +- *(5) Project [state_name#72, __pivot_count(1) AS `count(1)`#380[0] AS False#381L, __pivot_count(1) AS `count(1)`#380[1] AS True#382L]
      +- HashAggregate(keys=[state_name#72], functions=[pivotfirst(churn#38, count(1)#374L, False, True, 0, 0)])
         +- Exchange hashpartitioning(state_name#72, 200), ENSURE_REQUIREMENTS, [plan_id=821]
            +- HashAggregate(keys=[state_name#72], functions=[partial_pivotfirst(churn#38, count(1)#374L, False, True, 0, 0)])
               +- *(4) HashAggregate(keys=[state_name#72, churn#38], functions=[count(1)])
                  +- Exchange hashpartitioning(state_name#72, churn#38, 200), ENSURE_REQUIREMENTS, [plan_id=816]
                     +- *(3) HashAggregate(keys=[state_name#72, churn#38], functions=[partial_count(1)])
                        +- *(3) Project [chu

### Оптимизация 2

Используем **bucketBy**, чтобы избежать дополнительных операций сортировки и перетасовки при соединении датасетов

In [11]:
churn_df.repartition(1) \
    .write \
    .mode("overwrite") \
    .bucketBy(100, col="area code") \
    .option("path", "s3a://input/data/buckets/churn_df_bucket") \
    .saveAsTable("churn_df_bucket")

In [12]:
pop_dict \
    .withColumn("area code", F.col("area code").cast("string")) \
    .repartition(1) \
    .write \
    .bucketBy(100, "area code") \
    .option("path", "s3a://input/data/buckets/pop_dict_bucket") \
    .saveAsTable("pop_dict_bucket")

In [13]:
churn_df_bucket = spark.table("churn_df_bucket")
pop_dict_bucket = spark.table("pop_dict_bucket")

filtered_buckets = churn_df_bucket.join(pop_dict_bucket, on="area code", how="inner") \
    .filter(F.col("population") > 10000)

result = filtered_buckets.join(state_dict, filtered_buckets.state==state_dict.state_id, how="left") \
    .groupBy("state_name") \
    .pivot("churn") \
    .agg(F.count("*")) \
    .orderBy("state_name")

In [14]:
result.show(truncate=False)

+--------------------+-----+----+
|state_name          |False|True|
+--------------------+-----+----+
|Alabama             |58   |7   |
|Alaska              |44   |3   |
|Arizona             |53   |4   |
|Arkansas            |35   |9   |
|California          |21   |8   |
|Colorado            |49   |7   |
|Connecticut         |56   |8   |
|Delaware            |48   |8   |
|District of Columbia|43   |4   |
|Florida             |45   |8   |
|Georgia             |42   |8   |
|Hawaii              |42   |3   |
|Idaho               |53   |9   |
|Illinois            |49   |5   |
|Indiana             |59   |9   |
|Iowa                |34   |3   |
|Kansas              |50   |12  |
|Kentucky            |51   |8   |
|Louisiana           |47   |4   |
|Maine               |49   |13  |
+--------------------+-----+----+
only showing top 20 rows



# Задание 2

## Входные данные

*skew_transactions.csv* - информация о длительности просомтра контента пользователям
колонки:
1. user_uid — уникальный идентификатор пользователя
2. element_uid — уникальный идентификатор контента
3. watched_time — время просмотра в секундах

*catalogue.json* - каталог с описанием контента и метаинформации по нему
колонки:
1. type — тип элемента
2. duration — длительность в минутах (средняя длительность эпизода в случае с сериалами и многосерийными фильмами), округлённая до десятков
3. attributes — анонимизированные атрибуты данного элемента
4. availability — доступные права на элемент(subscription, purchase, rent)
5. feature_1 — анонимизированная вещественная переменная
6. feature_2 — анонимизированная вещественная переменная
7. feature_3 — анонимизированная порядковая переменная
8. feature_4 — анонимизированная вещественная переменная
9. feature_5 — анонимизированная вещественная переменная

## Что нужно сделать
1. Выполните join основных данных со справочником используя DataFrame API (по колонке id для контента - `element_uid`)
2. Описать проблему в датасетах с точки зрения обработки Spark
3. Решить задачу любым способом
4. Решить задачу с помощью salt-join подхода

P.S. Как вы можете заметить при просмотре данных по пользователями, нужный нам ключ для операции будет перекошен (90% строк представлены на фильм, очень популярный среди смотревших) - это нужно доказать в рамках п.2

### Решение

In [15]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd

In [17]:
skew_transactions_df = spark.read.option("header", True).csv("s3a://input/data/skew_transactions.csv")
catalogue_df = pd.read_json("catalogue.json", dtype=str)

In [18]:
catalogue_df = catalogue_df.transpose().reset_index()
catalogue_df = catalogue_df.rename(columns={"index": "element_uid"})

In [19]:
catalogue_df = spark.createDataFrame(catalogue_df)
catalogue_df

DataFrame[element_uid: bigint, type: string, availability: string, duration: string, feature_1: string, feature_2: string, feature_3: string, feature_4: string, feature_5: string, attributes: string]

In [20]:
result = skew_transactions_df.join(catalogue_df.select("element_uid", "type", "availability", "duration"), "element_uid", "left")

In [21]:
# Здесь необходимо вывести результат:
result.explain()
result.show(truncate=False)

== Physical Plan ==
*(4) Project [element_uid#706, user_uid#707, watched_time#708, type#713, availability#714, duration#715]
+- *(4) SortMergeJoin [cast(element_uid#706 as bigint)], [element_uid#712L], LeftOuter
   :- *(1) Sort [cast(element_uid#706 as bigint) ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(cast(element_uid#706 as bigint), 200), ENSURE_REQUIREMENTS, [plan_id=1510]
   :     +- FileScan csv [element_uid#706,user_uid#707,watched_time#708] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[s3a://input/data/skew_transactions.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<element_uid:string,user_uid:string,watched_time:string>
   +- *(3) Sort [element_uid#712L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(element_uid#712L, 200), ENSURE_REQUIREMENTS, [plan_id=1521]
         +- *(2) Project [element_uid#712L, type#713, availability#714, duration#715]
            +- *(2) Filter isnotnull(element_ui

24/06/27 20:06:49 WARN TaskSetManager: Stage 37 contains a task of very large size (1035 KiB). The maximum recommended task size is 1000 KiB.


+-----------+--------+------------+-----+------------------------------------+--------+
|element_uid|user_uid|watched_time|type |availability                        |duration|
+-----------+--------+------------+-----+------------------------------------+--------+
|6130       |563180  |3264        |movie|['purchase', 'rent']                |90      |
|2714       |408794  |7583        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |247546  |4450        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |43556   |7569        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |169048  |7577        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |542158  |8834        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |356858  |7763        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |277533  |4781        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |345703  |7639      

**Проблема**:
Есть перекос в датасете skew_transactions.csv, в результате чего наиболее частые рабочие узлы будут перегружены, так как получат больше данных для обработки. Также перегруженные узлы могут привести к тому, что у нас не останется доступной памяти и придется использовать, например, дисковое пространство, что скажется на скорости выполнения операций. 

### Решение с оптимизацией

In [22]:
# Используем salt join 
skew_transactions_df = skew_transactions_df.withColumn("salt", (F.rand() * 5).cast("int"))
catalogue_df= catalogue_df.withColumn("salt", (F.rand() * 5).cast("int"))

skew_transactions_df = skew_transactions_df.repartition(5, "element_uid", "salt")
catalogue_df = catalogue_df.repartition(5, "element_uid", "salt")

In [23]:
result = skew_transactions_df.join(catalogue_df.select("element_uid", "type", "availability", "duration", "salt"), ["element_uid", "salt"], "left")
grouped_result = result.groupBy("element_uid", "salt").agg(F.count("*").alias("count"))
result = grouped_result.select("element_uid", "salt", "count")

In [24]:
# Здесь необходимо вывести результат:
result.explain()
result.show(truncate=False)

== Physical Plan ==
*(6) HashAggregate(keys=[element_uid#706, salt#771], functions=[count(1)])
+- Exchange hashpartitioning(element_uid#706, salt#771, 200), ENSURE_REQUIREMENTS, [plan_id=1680]
   +- *(5) HashAggregate(keys=[element_uid#706, salt#771], functions=[partial_count(1)])
      +- *(5) Project [element_uid#706, salt#771]
         +- *(5) SortMergeJoin [cast(element_uid#706 as bigint), salt#771], [element_uid#712L, salt#776], LeftOuter
            :- *(2) Sort [cast(element_uid#706 as bigint) ASC NULLS FIRST, salt#771 ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(cast(element_uid#706 as bigint), salt#771, 200), ENSURE_REQUIREMENTS, [plan_id=1665]
            :     +- Exchange hashpartitioning(element_uid#706, salt#771, 5), REPARTITION_BY_NUM, [plan_id=1664]
            :        +- *(1) Project [element_uid#706, cast((rand(-3466626343010116864) * 5.0) as int) AS salt#771]
            :           +- FileScan csv [element_uid#706] Batched: false, DataFilte

24/06/27 20:07:10 WARN TaskSetManager: Stage 50 contains a task of very large size (1035 KiB). The maximum recommended task size is 1000 KiB.


+-----------+----+-----+
|element_uid|salt|count|
+-----------+----+-----+
|2692       |1   |57   |
|7789       |2   |46   |
|4886       |4   |71   |
|3303       |3   |45   |
|8408       |4   |46   |
|2522       |4   |36   |
|3023       |1   |37   |
|5694       |2   |50   |
|9971       |0   |60   |
|6105       |1   |55   |
|9876       |2   |54   |
|9766       |4   |62   |
|4401       |4   |57   |
|4888       |0   |40   |
|9794       |4   |38   |
|9404       |1   |64   |
|8968       |4   |32   |
|9138       |0   |64   |
|7252       |4   |43   |
|2991       |2   |55   |
+-----------+----+-----+
only showing top 20 rows



# Задание 3

## Входные данные

*cut_transactions.csv*  — информация о длительности просомтра контента пользователям

Описание фичей в cut_transactions.csv:
1. user_uid — уникальный идентификатор пользователя
2.  element_uid — уникальный идентификатор контента
3.  watched_time — время просмотра в секундах

*cut_ratings.csv*  — информация об оценках, поставленных пользователями

Описание фичей в cut_ratings.csv:
1. user_uid — уникальный идентификатор пользователя
2. element_uid — уникальный идентификатор контента
3. rating — поставленный пользователем рейтинг

*ids.csv*  — выборка пользователей
Описание фичей в ids.csv:
1. user_uid — уникальный идентификатор пользователя


## Что нужно сделать
Для каждого пользователя из выборки посчитать:
1. Максимальное и минимальное время просмотра фильмов с оценками 8, 9 и 10
2. Название фичи должно быть в формате feat_агрегирующая_функция_watched_time_rating_оценка.
3. Если у пользователь не ставил оценки 8, 9 и 10 то значение фичей должно быть null
4. Описать принятые при разработки кода решения и возможные оптимизации

P.S. На каждом этапе обработки должно быть должны агрегироваться минимально возможные объемы данных (сокращаем затраты на shuflle)

### Решение

In [25]:
transactions_df = spark.read.option("header", True).csv("s3a://input/data/cut_transactions.csv")
ratings_df = spark.read.option("header", True).csv("s3a://input/data/cut_ratings.csv")
ids_df = spark.read.option("header", True).csv("s3a://input/data/ids.csv")

In [26]:
transactions_df, ratings_df, ids_df

(DataFrame[element_uid: string, user_uid: string, watched_time: string],
 DataFrame[user_uid: string, element_uid: string, rating: string],
 DataFrame[user_uid: string])

In [27]:
combined_data = transactions_df.join(ratings_df, ["user_uid", "element_uid"], "inner").join(F.broadcast(ids_df), "user_uid", "inner")

aggregated_data = combined_data.groupBy("user_uid").agg(
    F.max(F.when(F.col("rating") == 8, F.col("watched_time"))).alias("max_watch_time_score_8"),
    F.min(F.when(F.col("rating") == 8, F.col("watched_time"))).alias("min_watch_time_score_8"),
    F.max(F.when(F.col("rating") == 9, F.col("watched_time"))).alias("max_watch_time_score_9"),
    F.min(F.when(F.col("rating") == 9, F.col("watched_time"))).alias("min_watch_time_score_9"),
    F.max(F.when(F.col("rating") == 10, F.col("watched_time"))).alias("max_watch_time_score_10"),
    F.min(F.when(F.col("rating") == 10, F.col("watched_time"))).alias("min_watch_time_score_10")
)

result = F.broadcast(ids_df).join(aggregated_data, "user_uid", "left")

result.write.csv("s3a://output/data/aggregated_data.csv", header=True)

24/06/27 20:07:47 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for left outer join.
24/06/27 20:07:47 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [28]:
# Здесь необходимо вывести результат:
result.explain()
result.show(truncate=False)

24/06/27 20:07:52 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for left outer join.
24/06/27 20:07:52 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for left outer join.


== Physical Plan ==
*(9) Project [user_uid#894, max_watch_time_score_8#912, min_watch_time_score_8#914, max_watch_time_score_9#916, min_watch_time_score_9#918, max_watch_time_score_10#920, min_watch_time_score_10#922]
+- *(9) SortMergeJoin [user_uid#894], [user_uid#849], LeftOuter
   :- *(1) Sort [user_uid#894 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(user_uid#894, 200), ENSURE_REQUIREMENTS, [plan_id=2166]
   :     +- FileScan csv [user_uid#894] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[s3a://input/data/ids.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_uid:string>
   +- SortAggregate(key=[user_uid#849], functions=[max(CASE WHEN (cast(rating#873 as int) = 8) THEN watched_time#850 END), min(CASE WHEN (cast(rating#873 as int) = 8) THEN watched_time#850 END), max(CASE WHEN (cast(rating#873 as int) = 9) THEN watched_time#850 END), min(CASE WHEN (cast(rating#873 as int) = 9) THEN watched_time#850 END), max(

Был использован **broadcast has join**, так как таблицы ids.csv и cut_ratings.csv небольшие (т.к. < 10 мб). 